In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
import cv2

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
test_csv = pd.read_csv("/kaggle/input/gtsrb-german-traffic-sign/Test.csv",skiprows = 0)
train_csv = pd.read_csv("/kaggle/input/gtsrb-german-traffic-sign/Test.csv",skiprows = 0)


IMG_WIDTH = 30
IMG_HEIGHT = 30

In [ ]:
test_images = []
test_labels = []

label_coll = set()

for _,row in test_csv.iterrows():
    filename = row["Path"]
    label = row["ClassId"]
    path = f"/kaggle/input/gtsrb-german-traffic-sign/{filename}"
    img = cv2.imread(path, 1)
    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
    test_images.append(img)
    test_labels.append(int(label))
    label_coll.add(label)

test_labels = tf.keras.utils.to_categorical(test_labels)

In [ ]:
train_images = []
train_labels = []

for _,row in train_csv.iterrows():
    filename = row["Path"]
    label = row["ClassId"]
    path = f"/kaggle/input/gtsrb-german-traffic-sign/{filename}"
    img = cv2.imread(path, 1)
    img = cv2.resize(img, (IMG_WIDTH, IMG_HEIGHT))
    train_images.append(img)
    train_labels.append(int(label))

train_labels = tf.keras.utils.to_categorical(train_labels)

In [ ]:
num_labels = len(label_coll)

DROPOUT_RATE_SP = 0.3
DROPOUT_RATE = 0.3
BATCH_SIZE = 5


In [ ]:
model = tf.keras.Sequential()

# initialize with He Normal
initializer = tf.keras.initializers.HeNormal()

#Input layer block convolution - batch normalization - dropout - relu
model.add(tf.keras.layers.Conv2D(filters = 30, kernel_size = 3, kernel_initializer=initializer, input_shape=(30, 30, 3)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.SpatialDropout2D(rate = DROPOUT_RATE_SP))
model.add(tf.keras.layers.Activation('relu'))

# block 2 downsampling - batch normalization - dropout - relu
model.add(tf.keras.layers.Conv2D(filters = 30, kernel_size = 2, strides = 2, kernel_initializer = initializer))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.SpatialDropout2D(rate = DROPOUT_RATE_SP))
model.add(tf.keras.layers.Activation('relu'))

# block 3 convolution - batch normalization - dropout - relu
model.add(tf.keras.layers.Conv2D(filters = 30, kernel_size = 3, kernel_initializer=initializer))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.3))
model.add(tf.keras.layers.Activation('relu'))

# block 4 downsampling - batch normalization - dropout - relu
model.add(tf.keras.layers.Conv2D(filters = num_labels, kernel_size = 2, strides = 2, kernel_initializer=initializer))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(DROPOUT_RATE))
model.add(tf.keras.layers.Activation('relu'))

# block 5 convolution - batch normalization - dropout - relu
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(filters = num_labels, kernel_size = 3, kernel_initializer=initializer))
model.add(tf.keras.layers.Dropout(DROPOUT_RATE))
model.add(tf.keras.layers.Activation('relu'))

# block 6 downsampling - batch normalization - dropout - relu
model.add(tf.keras.layers.Conv2D(filters = num_labels, kernel_size = 2, strides = 2, kernel_initializer=initializer))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(DROPOUT_RATE))
model.add(tf.keras.layers.Activation('relu'))

# block 7 downsampling - batch normalization - relu
model.add(tf.keras.layers.Conv2D(filters = num_labels, kernel_size = 2, strides = 2, kernel_initializer=initializer))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Activation('relu'))

# output layer 1-convolution - softmax
model.add(tf.keras.layers.Conv2D(filters = num_labels, kernel_size = 1))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Softmax())

In [ ]:
 # show summary of layers and corresponding outputs
model.summary()

# optimizer
optimizer = tf.keras.optimizers.Adagrad(learning_rate=0.02, initial_accumulator_value=0.1, epsilon=1e-06)

model.compile(
optimizer=optimizer,
loss="categorical_crossentropy",
metrics=["accuracy"]
)

In [ ]:
max_accuracy = 0
count_lower = 0

while (True):
        # Fit model on training data
        model.fit(np.asarray(train_images), np.asarray(train_labels), epochs=5, batch_size = BATCH_SIZE)

        # Evaluate neural network performance
        accuracy = model.evaluate(np.asarray(test_images),  np.asarray(test_labels), verbose=2)[1]
        if accuracy > max_accuracy:
            count_lower = 0
            model.save("gtsrb_fcn")
            max_accuracy = accuracy
        else: 
            if count_lower > 4: 
                break
            count_lower += 1
            